# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])
import nltk
nltk.download('averaged_perceptron_tagger')

import re
import pickle
import pandas as pd
import numpy  as np
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem     import WordNetLemmatizer
from sqlalchemy    import create_engine

from sklearn.pipeline                import Pipeline, FeatureUnion
from sklearn.multioutput             import MultiOutputClassifier
from sklearn.ensemble                import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection         import train_test_split, GridSearchCV
from sklearn.metrics                 import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats.mstats              import gmean
from sklearn.base                    import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haiqi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haiqi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\haiqi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response_ETL.db')
df = pd.read_sql_table('message_categories', engine)
X = df['message']
y = df.iloc[:,4:]

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
category_names = y.columns
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect',  CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',   MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [8]:
#Train pipeline
pipeline.fit(X_train, y_train)

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

def scoring_model(y_true, y_pred):
    multioutput_f1   = multioutput_fscore(y_true, y_pred, beta = 1)
    overall_accuracy = (y_pred == y_true).mean().mean()

    print('Average overall accuracy:     {0:.2f}%\n'.format(overall_accuracy*100))
    print('F1 score (custom definition): {0:.2f}%\n'.format(multioutput_f1*100))
    
    for i in range(len(category_names)):
        print('------------------------------------------------------\n')
        print("Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i], y_pred[:, i]))


scorer = make_scorer(multioutput_fscore,greater_is_better = True)

In [10]:
y_pred = pipeline.predict(X_test)
scoring_model(y_test, y_pred)

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Average overall accuracy:     94.13%

F1 score (custom definition): 92.25%

------------------------------------------------------

Category: related 
               precision    recall  f1-score   support

           0       0.60      0.36      0.45      1510
           1       0.82      0.93      0.87      4993
           2       0.46      0.12      0.19        51

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.63      0.47      0.50      6554
weighted avg       0.77      0.79      0.77      6554

------------------------------------------------------

Category: request 
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      5409
           1       0.82      0.40      0.54      1145

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.85      0.69      0.73      6554
weighted avg       0.87      0.88      0.86      6554

------------------------------------------------------

Category

Category: fire 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6483
           1       1.00      0.01      0.03        71

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554

------------------------------------------------------

Category: earthquake 
               precision    recall  f1-score   support

           0       0.96      0.99      0.97      5934
           1       0.89      0.56      0.68       620

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.92      0.77      0.83      6554
weighted avg       0.95      0.95      0.95      6554

------------------------------------------------------

Category: cold 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6424
           1       0.73      0.08      0.15       130

   micro avg 

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {  'vect__ngram_range':  [(1,1),(1,2)],
                'vect__max_df':       (0.5, 0.75, 1.0),
                'vect__max_features': (None, 5000, 10000),
                'tfidf__use_idf':     (True, False),
             }

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer,verbose = 2, n_jobs = -1)
cv.fit(X_train, y_train)

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 14.4min finished
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2)], 'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (None, 5000, 10000), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(multioutput_fscore), verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_pred = cv.predict(X_test)
scoring_model(y_test, y_pred)

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Average overall accuracy:     94.66%

F1 score (custom definition): 93.17%

------------------------------------------------------

Category: related 
               precision    recall  f1-score   support

           0       0.63      0.40      0.49      1510
           1       0.83      0.93      0.88      4993
           2       0.38      0.18      0.24        51

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.61      0.50      0.54      6554
weighted avg       0.78      0.80      0.78      6554

------------------------------------------------------

Category: request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5409
           1       0.83      0.46      0.59      1145

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.86      0.72      0.76      6554
weighted avg       0.88      0.89      0.88      6554

------------------------------------------------------

Category

Category: floods 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6008
           1       0.89      0.41      0.56       546

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.92      0.70      0.77      6554
weighted avg       0.94      0.95      0.94      6554

------------------------------------------------------

Category: storm 
               precision    recall  f1-score   support

           0       0.95      0.98      0.97      5983
           1       0.72      0.50      0.59       571

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.84      0.74      0.78      6554
weighted avg       0.93      0.94      0.93      6554

------------------------------------------------------

Category: fire 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6483
           1       1.00      0.01      0.03        71

   micro avg    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# StartingVerbExtractor Class Definition
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [14]:
pipeline = Pipeline([('features', FeatureUnion([('text_pipeline', Pipeline([('vect',  CountVectorizer(tokenizer=tokenize)),
                                                                            ('tfidf', TfidfTransformer()),
                                                                          ])),
                                                ('starting_verb', StartingVerbExtractor())
                                              ])
                     ),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                   ])

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

In [22]:
y_pred = pipeline.predict(X_test)
scoring_model(y_test, y_pred)

Average overall accuracy:     94.66%

F1 score (custom definition): 93.52%

------------------------------------------------------

Category: related 
              precision    recall  f1-score   support

          0       0.58      0.24      0.34      1511
          1       0.80      0.95      0.87      4992
          2       0.26      0.12      0.16        51

avg / total       0.75      0.78      0.74      6554

------------------------------------------------------

Category: request 
              precision    recall  f1-score   support

          0       0.91      0.96      0.94      5400
          1       0.77      0.54      0.64      1154

avg / total       0.88      0.89      0.88      6554

------------------------------------------------------

Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.20      0.03      0.05        32

avg / total       0.99      0.99      0.99      6554



In [16]:
#Use grid search to find better parameters
parameters = {
        'features__text_pipeline__vect__ngram_range':  ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df':       (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf':     (True, False),
}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer,verbose = 2, n_jobs = -1)
cv.fit(X_train, y_train)

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 20.3min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 75.9min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0), 'features__text_pipeline__vect__max_features': (None, 5000, 10000), 'features__text_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(multioutput_fscore), verbose=2)

In [17]:
y_pred = cv.predict(X_test)
scoring_model(y_test, y_pred)

Average overall accuracy:     94.71%

F1 score (custom definition): 93.58%

------------------------------------------------------

Category: related 
               precision    recall  f1-score   support

           0       0.54      0.21      0.31      1510
           1       0.79      0.94      0.86      4993
           2       0.35      0.16      0.22        51

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.56      0.44      0.46      6554
weighted avg       0.73      0.77      0.73      6554

------------------------------------------------------

Category: request 
               precision    recall  f1-score   support

           0       0.91      0.97      0.94      5409
           1       0.77      0.56      0.65      1145

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.84      0.76      0.79      6554
weighted avg       0.89      0.89      0.89      6554

------------------------------------------------------

Category

               precision    recall  f1-score   support

           0       0.89      0.95      0.92      4762
           1       0.84      0.67      0.75      1792

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.86      0.81      0.83      6554
weighted avg       0.87      0.88      0.87      6554

------------------------------------------------------

Category: floods 
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      6008
           1       0.89      0.55      0.68       546

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.93      0.77      0.83      6554
weighted avg       0.96      0.96      0.95      6554

------------------------------------------------------

Category: storm 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      5983
           1       0.70      0.56      0.62       571

   micro avg       0.94      0.9

### 9. Export your model as a pickle file

In [18]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.